In [1]:
from models.lbanp import LBANP
from models.tnpnd import TNPND

In [2]:

import yaml
import os.path as osp
import torch

name = "lbanp"
device = "cpu"


# lbanp_model = LBANP(num_latents=8,
#                 dim_x= 2,
#                 dim_y= 5,
#                 d_model=16,
#                 emb_depth=3,
#                 dim_feedforward=64,
#                 nhead=1,
#                 dropout=0.0,
#                 num_layers=4,
#                 drop_y=0.5,
#                 device="cpu")

with open(osp.join("configs", "wheel", f"{name}.yaml")) as g:
    config = yaml.safe_load(g)

path = osp.join("results", "wheel", "train-all-R", "lbanp", "S0-B8", "lbanp-num_latents-8")
filename = 'ckpt.tar'

model = LBANP(**config).to(device)
model.device = torch.device(device)
model.eval()
file = osp.join(path, filename)
if not osp.exists(file):
    raise FileNotFoundError(file)
else:
    ckpt = torch.load(file)
    model.load_state_dict(ckpt.model)
# tnp_model = TNPND(
#         dim_x= 2,
#                 dim_y= 5,
#                 d_model=16,
#                 emb_depth=3,
#                 dim_feedforward=64,
#                 nhead=1,
#                 dropout=0.0,
#                 num_layers=4,
#                 num_std_layers=2,
#                 drop_y=0.5,
#                 cov_approx='lowrank', # cholesky or lowrank parameterization
#                 prj_dim=20,
#                 prj_depth=4,
#                 diag_depth=4,
#     device="cpu"
#     )

In [3]:
from data.wheel import sample_wheel_data, WheelBanditSampler

In [4]:
sampler = WheelBanditSampler()

In [5]:
batch = sampler.sample(
            batch_size=4,
            device="cpu",
            seed=0,
            reward="all",
            delta=0.9
        )

In [6]:
pred = model.predict(batch.xc, batch.yc, batch.xt)

In [7]:
pred.loc

tensor([[[ 1.2149, 32.0858,  0.9722,  0.9951,  0.9732],
         [ 1.2004,  4.6976,  0.9989,  1.0036,  0.9882],
         [ 1.2028,  1.0039, 35.8573,  1.0025,  0.9916],
         [ 1.1959,  1.0051,  0.9920, 48.3353,  0.9867],
         [ 1.2008,  1.0021,  0.9994,  1.0060,  1.0242],
         [ 1.2033,  6.4271,  0.9897,  0.9985,  0.9846],
         [ 1.2009, 44.7398,  1.0000,  1.0024,  0.9963],
         [ 1.2011,  1.0029,  0.9993,  1.0059,  1.0300],
         [ 1.2053,  1.0333,  1.0284,  1.0124, 20.8124],
         [ 1.2068,  1.0019, 32.6120,  1.0061,  0.9897],
         [ 1.2074,  1.0104,  0.9958,  0.9974, 40.7438],
         [ 1.2011,  1.0036,  0.9912, 48.9159,  1.0141],
         [ 1.2046,  1.0009,  0.9979,  0.9996,  1.0008],
         [ 1.1949,  0.9897,  0.9813, 49.9804,  0.9978],
         [ 1.1950,  0.9998,  0.9933, 48.8795,  0.9871],
         [ 1.1954,  0.9852, 12.1634,  0.9903,  0.9998],
         [ 1.1993,  1.0031,  0.9880, 49.0633,  1.0144],
         [ 1.1995,  1.0034,  0.9853, 49.3116,  1

In [8]:
ar_pred = model.ar_predict(batch.xc, batch.yc, batch.xt)

In [9]:
ar_pred.loc

tensor([[[ 1.2132, 36.1363,  0.9823,  0.9970,  0.9778],
         [ 1.2067, 12.5654,  0.9912,  0.9999,  0.9899],
         [ 1.2000,  1.0017, 37.6298,  0.9984,  0.9965],
         [ 1.1944,  1.0036,  0.9928, 48.4789,  0.9879],
         [ 1.2008,  1.0045,  1.0007,  1.0061,  1.0189],
         [ 1.2107, 22.5964,  0.9872,  0.9959,  0.9872],
         [ 1.2008, 44.7468,  1.0000,  1.0024,  0.9963],
         [ 1.2011,  1.0029,  0.9993,  1.0059,  1.0298],
         [ 1.1966,  1.0255,  1.0359,  1.0103, 33.9245],
         [ 1.2043,  1.0031, 34.9783,  1.0044,  0.9967],
         [ 1.2061,  1.0096,  0.9909,  0.9996, 43.4366],
         [ 1.1986,  1.0058,  0.9878, 49.0488,  1.0154],
         [ 1.2046,  1.0020,  0.9994,  0.9995,  1.0024],
         [ 1.1933,  0.9899,  0.9847, 49.9823,  0.9991],
         [ 1.1947,  0.9983,  0.9930, 48.9710,  0.9873],
         [ 1.1949,  0.9845, 12.5784,  0.9854,  0.9996],
         [ 1.1991,  1.0026,  0.9842, 49.4855,  1.0097],
         [ 1.1995,  1.0034,  0.9851, 49.3309,  1

In [11]:
batch.xc.shape

torch.Size([8, 512, 2])

In [12]:
batch.yc.shape

torch.Size([8, 512, 5])

In [13]:
batch.xt.shape

torch.Size([8, 50, 2])

In [14]:
batch.yt.shape

torch.Size([8, 50, 5])

In [7]:
ll_loss, pred = lbanp_model(batch)

In [14]:
tnp_model(batch)

AttrDict({'loss_target': tensor(666.4536, grad_fn=<NegBackward0>), 'loss_ctx': tensor(1781.4780, grad_fn=<MeanBackward0>), 'loss': tensor(2447.9316, grad_fn=<AddBackward0>), 'mean_std': tensor(0.0412, grad_fn=<MeanBackward0>), 'rmse': tensor(336.7043, grad_fn=<MeanBackward0>)})

In [17]:
pred.sample(torch.Size([10]))[1]

tensor([[[-0.4370, -1.0098,  0.1075, -0.9991,  0.4572],
         [-0.1563, -0.3379,  0.4788,  0.1298,  0.6714],
         [ 0.2853, -0.2805,  0.3360,  0.8649, -0.9022],
         ...,
         [ 1.2777,  0.1652,  0.6480,  0.2400,  1.1537],
         [ 0.7377,  0.2597,  1.0902, -0.0205, -0.4204],
         [ 0.1903, -0.3400,  0.1030, -0.1797,  0.2685]],

        [[-0.3725,  0.3566, -1.8452,  0.7515,  0.4054],
         [ 0.6287, -0.5361,  1.3953,  1.1158, -0.8499],
         [-0.5555, -0.6780,  3.5492,  0.5310,  1.0871],
         ...,
         [ 1.2121,  0.1738,  2.3070, -1.4268,  1.1481],
         [ 1.1972, -0.1266,  0.5311, -0.2256,  0.8538],
         [ 1.8189, -0.7754, -0.6902, -0.3359, -0.3103]],

        [[ 0.8968,  0.4521,  0.4219, -1.3565, -0.3342],
         [-1.7084,  0.4685,  1.7808,  0.9149,  0.6502],
         [ 1.3056, -1.0105, -0.4376,  0.3604,  2.1575],
         ...,
         [ 0.1437, -1.3362, -0.9561,  0.4158,  0.5472],
         [ 1.8388, -0.7121, -1.5469, -0.1130, -0.5621],
  

In [23]:
batch.yc.shape

torch.Size([8, 512, 5])

In [8]:
import numpy as np
a = np.random.randn(3,2,2)

In [9]:
a

array([[[ 0.21669559, -0.74235375],
        [-0.16376122, -0.70650212]],

       [[-0.53418005,  0.00455652],
        [ 0.57478282,  0.07178991]],

       [[ 0.75321925,  0.30662907],
        [ 0.62068709,  0.79254104]]])

In [10]:
a.reshape(-1,2).reshape(3,2,2)

array([[[ 0.21669559, -0.74235375],
        [-0.16376122, -0.70650212]],

       [[-0.53418005,  0.00455652],
        [ 0.57478282,  0.07178991]],

       [[ 0.75321925,  0.30662907],
        [ 0.62068709,  0.79254104]]])